In [ ]:
import cv2
import numpy as np
import getopt
import sys
import random

In [ ]:
def readImage(filename):
    img = cv2.imread(filename, 0)
    if img is None:
        print('Invalid image:' + filename)
        return None
    else:
        print('Image successfully read...')
        return img

In [ ]:
# This draws matches and optionally a set of inliers in a different color
# Note: I lifted this drawing portion from stackoverflow and adjusted it to my needs because OpenCV 2.4.11 does not
# include the drawMatches function
def drawMatches(img1, kp1, img2, kp2, matches, inliers = None):
    # Create a new output image that concatenates the two images together
    rows1 = img1.shape[0]
    cols1 = img1.shape[1]
    rows2 = img2.shape[0]
    cols2 = img2.shape[1]

    out = np.zeros((max([rows1,rows2]),cols1+cols2,3), dtype='uint8')

    # Place the first image to the left
    out[:rows1,:cols1,:] = np.dstack([img1, img1, img1])

    # Place the next image to the right of it
    out[:rows2,cols1:cols1+cols2,:] = np.dstack([img2, img2, img2])

    # For each pair of points we have between both images
    # draw circles, then connect a line between them
    for mat in matches:

        # Get the matching keypoints for each of the images
        img1_idx = mat.queryIdx
        img2_idx = mat.trainIdx

        # x - columns, y - rows
        (x1,y1) = kp1[img1_idx].pt
        (x2,y2) = kp2[img2_idx].pt

        inlier = False

        if inliers is not None:
            for i in inliers:
                if i.item(0) == x1 and i.item(1) == y1 and i.item(2) == x2 and i.item(3) == y2:
                    inlier = True

        # Draw a small circle at both co-ordinates
        cv2.circle(out, (int(x1),int(y1)), 4, (255, 0, 0), 1)
        cv2.circle(out, (int(x2)+cols1,int(y2)), 4, (255, 0, 0), 1)

        # Draw a line in between the two points, draw inliers if we have them
        if inliers is not None and inlier:
            cv2.line(out, (int(x1),int(y1)), (int(x2)+cols1,int(y2)), (0, 255, 0), 1)
        elif inliers is not None:
            cv2.line(out, (int(x1),int(y1)), (int(x2)+cols1,int(y2)), (0, 0, 255), 1)

        if inliers is None:
            cv2.line(out, (int(x1),int(y1)), (int(x2)+cols1,int(y2)), (255, 0, 0), 1)

    return out

In [ ]:
def findFeatures(img):
    sift = cv2.SIFT()
    keypoints, descriptors = sift.detectAndCompute(img, None)
    
    #img = cv2.drawKeypoints(img, keypoints)
    #cv2.imwrite('sift_keypoints.png', img)

    return keypoints, descriptors

In [ ]:
def matchFeatures(kp1, kp2, desc1, desc2, img1, img2):
    matcher = cv2.BFMatcher(cv2.NORM_L2, True)
    matches = matcher.match(desc1, desc2)
    #matchImg = drawMatches(img1,kp1,img2,kp2,matches)
    #cv2.imwrite('Matches.png', matchImg)
    return matches

In [ ]:
#
# Computers a homography from 4-correspondences
#
def calculateHomography(correspondences):
    #loop through correspondences and create assemble matrix
    aList = []
    for corr in correspondences:
        p1 = np.matrix([corr.item(0), corr.item(1), 1])
        p2 = np.matrix([corr.item(2), corr.item(3), 1])

        a2 = [0, 0, 0, -p2.item(2) * p1.item(0), -p2.item(2) * p1.item(1), -p2.item(2) * p1.item(2),
              p2.item(1) * p1.item(0), p2.item(1) * p1.item(1), p2.item(1) * p1.item(2)]
        a1 = [-p2.item(2) * p1.item(0), -p2.item(2) * p1.item(1), -p2.item(2) * p1.item(2), 0, 0, 0,
              p2.item(0) * p1.item(0), p2.item(0) * p1.item(1), p2.item(0) * p1.item(2)]
        aList.append(a1)
        aList.append(a2)
 
    matrixA = np.matrix(aList)

    #svd composition
    u, s, v = np.linalg.svd(matrixA)

    #reshape the min singular value into a 3 by 3 matrix
    h = np.reshape(v[8], (3, 3))

    #normalize and now we have h
    h = (1/h.item(8)) * h
    
    return h

In [ ]:
#
#Calculate the geometric distance between estimated points and original points
#
def geometricDistance(correspondence, h):

    p1 = np.transpose(np.matrix([correspondence[0].item(0), correspondence[0].item(1), 1]))
    estimatep2 = np.dot(h, p1)
    estimatep2 = (1/estimatep2.item(2))*estimatep2

    p2 = np.transpose(np.matrix([correspondence[0].item(2), correspondence[0].item(3), 1]))
    error = p2 - estimatep2
    return np.linalg.norm(error)

In [ ]:
#
#Runs through ransac algorithm, creating homographies from random correspondences
#
def ransac(corr, thresh):
    maxInliers = []
    finalH = None
    for i in range(1000):
        #find 4 random points to calculate a homography
        corr1 = corr[random.randrange(0, len(corr))]
        corr2 = corr[random.randrange(0, len(corr))]
        randomFour = np.vstack((corr1, corr2))
        corr3 = corr[random.randrange(0, len(corr))]
        randomFour = np.vstack((randomFour, corr3))
        corr4 = corr[random.randrange(0, len(corr))]
        randomFour = np.vstack((randomFour, corr4))

        #call the homography function on those points
        h = calculateHomography(randomFour)
        inliers = []

        for i in range(len(corr)):
            d = geometricDistance(corr[i], h)
            if d < 5:
                inliers.append(corr[i])

        if len(inliers) > len(maxInliers):
            maxInliers = inliers
            finalH = h
        print("Corr size: ", len(corr), " NumInliers: ", len(inliers), "Max inliers: ", len(maxInliers))

        if len(maxInliers) > (len(corr)*thresh):
            break
    return finalH, maxInliers

In [ ]:
# #
# # Main parses argument list and runs the functions
# #
# def main():
#     args, img_name = getopt.getopt(sys.argv[1:],'', ['threshold='])
#     args = dict(args)

#     estimation_thresh = args.get('--threshold')
#     print("Estimation Threshold: ", estimation_thresh)
#     if estimation_thresh is None:
#         estimation_thresh = 0.60

#     img1name = str(img_name[0])
#     img2name = str(img_name[1])
#     print("Image 1 Name: " + img1name)
#     print("Image 2 Name: " + img2name)

#     #query image
#     img1 = readImage(img_name[0])
#     #train image
#     img2 = readImage(img_name[1])

#     #find features and keypoints
#     correspondenceList = []
#     if img1 is not None and img2 is not None:
#         kp1, desc1 = findFeatures(img1)
#         kp2, desc2 = findFeatures(img2)
#         print("Found keypoints in " + img1name + ": " + str(len(kp1)))
#         print("Found keypoints in " + img2name + ": " + str(len(kp2)))
#         keypoints = [kp1, kp2]
#         matches = matchFeatures(kp1, kp2, desc1, desc2, img1, img2)
        
#         for match in matches:
#             (x1, y1) = keypoints[0][match.queryIdx].pt
#             (x2, y2) = keypoints[1][match.trainIdx].pt
#             correspondenceList.append([x1, y1, x2, y2])

#         corrs = np.matrix(correspondenceList)

#         #run ransac algorithm
#         finalH, inliers = ransac(corrs, estimation_thresh)
#         print("Final homography: ", finalH)
#         print("Final inliers count: ", len(inliers))

#         matchImg = drawMatches(img1,kp1,img2,kp2,matches,inliers)
#         cv2.imwrite('InlierMatches.png', matchImg)

#         f = open('homography.txt', 'w')
#         f.write("Final homography: \n" + str(finalH)+"\n")
#         f.write("Final inliers count: " + str(len(inliers)))
#         f.close()

In [ ]:
args, img_name = getopt.getopt(sys.argv[1:],'', ['threshold='])
args = dict(args)

estimation_thresh = args.get('--threshold')
print("Estimation Threshold: ", estimation_thresh)
if estimation_thresh is None:
    estimation_thresh = 0.60

img1name = str(img_name[0])
img2name = str(img_name[1])
print("Image 1 Name: " + img1name)
print("Image 2 Name: " + img2name)

#query image
img1 = readImage(img_name[0])
#train image
img2 = readImage(img_name[1])

#find features and keypoints
correspondenceList = []
if img1 is not None and img2 is not None:
    kp1, desc1 = findFeatures(img1)
    kp2, desc2 = findFeatures(img2)
    print("Found keypoints in " + img1name + ": " + str(len(kp1)))
    print("Found keypoints in " + img2name + ": " + str(len(kp2)))
    keypoints = [kp1, kp2]
    matches = matchFeatures(kp1, kp2, desc1, desc2, img1, img2)
    
    for match in matches:
        (x1, y1) = keypoints[0][match.queryIdx].pt
        (x2, y2) = keypoints[1][match.trainIdx].pt
        correspondenceList.append([x1, y1, x2, y2])

    corrs = np.matrix(correspondenceList)

    #run ransac algorithm
    finalH, inliers = ransac(corrs, estimation_thresh)
    print("Final homography: ", finalH)
    print("Final inliers count: ", len(inliers))

    matchImg = drawMatches(img1,kp1,img2,kp2,matches,inliers)
    cv2.imwrite('InlierMatches.png', matchImg)

    f = open('homography.txt', 'w')
    f.write("Final homography: \n" + str(finalH)+"\n")
    f.write("Final inliers count: " + str(len(inliers)))
    f.close()